In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [2]:
caminhos = [
    "../../assets/pdf/apostila.pdf",
    "../../assets/pdf/LLM.pdf",
    ]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i


Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [3]:
diretorio = 'arquivos/chat_retrieval_db'

embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)


In [4]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")


In [5]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
)


In [6]:
pergunta = "O que é Hugging Face e como faço para acessá-lo?"
chat_chain.invoke({"query": pergunta})


{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma empresa que desenvolve e mantém uma plataforma e uma biblioteca de modelos de aprendizado de máquina, especialmente focada em processamento de linguagem natural (NLP). A biblioteca mais conhecida é o Transformers, que permite o uso de modelos pré-treinados para tarefas como tradução, geração de texto e classificação.\n\nPara acessar o Hugging Face, você pode visitar o site oficial em [huggingface.co](https://huggingface.co), onde você encontrará uma variedade de modelos e ferramentas. Além disso, você pode instalar a biblioteca Transformers em seu ambiente Python usando o seguinte comando:\n\n```bash\npip install transformers\n```\n\nApós a instalação, você pode começar a usar os modelos disponíveis na biblioteca dentro do seu código Python.'}

In [8]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornecido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)


In [9]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents=True
)


In [10]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])


Hugging Face é uma plataforma que oferece uma biblioteca de modelos de aprendizado de máquina, especialmente para processamento de linguagem natural. Você pode acessá-lo através de seu site e utilizar seus modelos com suporte a frameworks de código aberto como o MLflow. É possível encontrar e integrar modelos em seu ambiente de programação, facilitando o uso para desenvolvedores.


In [11]:
resposta['source_documents']


[Document(page_content='Atualmente, requer um pouco mais de esforço para pegar um modelo de código aberto e começar a usá-lo, mas o progresso está ocorrendo muito rapidamente para torná-los mais acessíveis aos usuários. Na Databricks, por exemplo, fizemos melhorias em frameworks de código aberto como o MLflow para tornar muito fácil para alguém com um pouco de experiência em Python pegar qualquer modelo transformador da Hugging Face e usá-lo como um objeto Python. Muitas vezes, você pode encontrar um modelo de código aberto que resolve seu problema específico e que é várias ordens de grandeza menor que o ChatGPT, permitindo que você traga o modelo para seu ambiente e hospede-o você mesmo. Isso significa que você pode manter os dados sob seu controle para preocupações com privacidade e governança, além de gerenciar seus custos. Outra grande vantagem de usar modelos de código aberto é a capacidade de ajustá-los aos seus próprios dados', metadata={'doc_id': 75, 'page': 6, 'source': '../..

In [12]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Hugging Face e como faço para acessá-lo?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Hugging Face e como faço para acessá-lo?",
  "context": "Atualmente, requer um pouco mais de esforço para pegar um modelo de código aberto e começar a usá-lo, mas o progresso está ocorrendo muito rapidamente para torná-los mais acessíveis aos usuários. Na Databricks, por exemplo, fizemos melhorias em frameworks de código aberto como o MLflow para tornar muito fácil para alguém com um pouco de experiência em Python pegar qualquer modelo transformador da Hugging Face e usá-lo como um objeto Python. Muitas vezes, você pode encontrar um modelo de código aberto que resolve seu problema específico e que é várias ordens de grandeza

In [13]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])


Hugging Face é uma plataforma que oferece uma ampla gama de modelos de aprendizado de máquina, especialmente voltados para processamento de linguagem natural (NLP), incluindo Large Language Models (LLMs). Além disso, a Hugging Face também está se expandindo para outras áreas de IA generativa, como a geração de arte a partir de texto, áudio e vídeo, entre outras inovações que estão surgindo.

Para acessar o Hugging Face e explorar os LLMs e outros projetos de IA generativa, você pode seguir os seguintes passos:

1. **Visite o site**: Acesse o site oficial do Hugging Face em [huggingface.co](https://huggingface.co).

2. **Crie uma conta**: Você pode criar uma conta gratuita para acessar mais recursos e funcionalidades, como a possibilidade de salvar seus próprios modelos ou anotações.

3. **Explore os modelos**: No site, você pode navegar pela biblioteca de modelos disponíveis, onde pode filtrar por tarefa, linguagem e tipo de modelo, incluindo LLMs e outros projetos de IA generativa.

4